In [1]:
import random

class Battleship:
    # following original rules: 5 total ships: carrier-5, battleship-4, cruiser-3, submarine-3, destroyer-2
    def __init__(self, size=8):
        self.size = size
        self.board = [[' ']*size for _ in range(size)]
        self.ships = {'Carrier': 5, 'Battleship': 4, 'Cruiser': 3, 'Submarine': 3, 'Destroyer': 2}
        self.enemy_board = [[' ']*size for _ in range(size)]
        self.probability_map = [[0] * size for _ in range(size)]
        self.place_ships()

    # Place the ships:
    # function will iterate over each ship type and size, randomly placing it either horizontally or vertically on the board.
    def place_ships(self):
        for ship, size in self.ships.items():
            placed = False
            while not placed:
                orientation = random.choice(['horizontal', 'vertical'])
                if orientation == 'horizontal':
                    row = random.randint(0, self.size - 1)
                    col = random.randint(0, self.size - size)
                    if all(self.board[row][col + i] == ' ' for i in range(size)):
                        for i in range(size):
                            self.board[row][col + i] = 'S'
                        placed = True
                else:
                    row = random.randint(0, self.size - size)
                    col = random.randint(0, self.size - 1)
                    if all(self.board[row + i][col] == ' ' for i in range(size)):
                        for i in range(size):
                            self.board[row + i][col] = 'S'
                        placed = True

    # Print the board
    # following original rules 8x8 grid board
    def print_board(self, board, show_ships=False):
        print("   " + " ".join(f"{i:2}" for i in range(self.size)))
        for i in range(self.size):
            row_label = chr(65 + i)
            row_content = [
                "  " if cell == 'S' and not show_ships else f"{cell:2}"
                for cell in board[i]
            ]
            print(f"{row_label}  " + " ".join(row_content))

    # Make a guess
    # this function will receive coordinations for a guess and check if the ship is present at that location on the board
    # then it updates the enemy baord accordingly, marking hits and misses and also updating the probability map
    def make_guess(self, row, col):
        if self.board[row][col] == 'S':
            self.enemy_board[row][col] = 'X'
            self.update_probability_map(row, col, hit=True)
            return True
        else:
            self.enemy_board[row][col] = 'M'
            self.update_probability_map(row, col, hit=False)
            return False

    # Update the probability map around hits to prioritize nearby cells
    def update_probability_map(self, row, col, hit):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        if hit:
            self.probability_map[row][col] = 0  # reset the hit cell probability
            for dr, dc in directions:
                for i in range(1, 3):  # Increase probabilities for cells up to 2 spaces away
                    nr, nc = row + dr * i, col + dc * i
                    if 0 <= nr < self.size and 0 <= nc < self.size and self.enemy_board[nr][nc] == ' ':
                        # Increase by more points closer to hit, fewer points further away
                        self.probability_map[nr][nc] += (3 - i)
        else:
            self.probability_map[row][col] = 0  # reset the miss cell probability

    # agent for improved guessing
    def ai_guess(self):
        max_prob = -1
        best_guess = None
        for r in range(self.size):
            for c in range(self.size):
                if self.enemy_board[r][c] == ' ' and self.probability_map[r][c] > max_prob:
                    max_prob = self.probability_map[r][c]
                    best_guess = (r, c)

        if best_guess:
            return best_guess
        else:
            while True:
                row, col = random.randint(0, self.size - 1), random.randint(0, self.size - 1)
                if self.enemy_board[row][col] == ' ':
                    return row, col

    # Play the game with reinforcement-based AI
    def play_game(self):
        game_over = False
        while not game_over:
            row, col = self.ai_guess()
            if self.make_guess(row, col):
                print(f"Hit at {chr(65 + row)}, {col}")
            else:
                print(f"Miss at {chr(65 + row)}, {col}")
            self.print_board(self.enemy_board)
            # Check if all ships are hit (game over condition)
            if all(self.enemy_board[r][c] == 'H' for r in range(self.size) for c in range(self.size) if self.board[r][c] == 'S'):
                game_over = True
                print("Game over, all ships sunk!")

In [ ]:
#testing out the game
game = Battleship()

print("inital ship placement on the board:")
game.print_board(game.board, show_ships=True)

game.play_game()

inital ship placement on the board:
    0  1  2  3  4  5  6  7
A           S  S  S       
B           S  S  S       
C           S  S  S       
D              S  S       
E              S          
F                 S  S  S 
G     S  S                
H                         
Miss at A, 0
    0  1  2  3  4  5  6  7
A  M                      
B                         
C                         
D                         
E                         
F                         
G                         
H                         
Miss at A, 1
    0  1  2  3  4  5  6  7
A  M  M                   
B                         
C                         
D                         
E                         
F                         
G                         
H                         
Miss at A, 2
    0  1  2  3  4  5  6  7
A  M  M  M                
B                         
C                         
D                         
E                         
F                         
G      